# Simple End to End

This part  demonstrates the basic end-to-end flow from an ONNX model to a generated hardware IP. We encourage you to run this example to try our automated toolflow, and then follow the rest of the tutorial which provides a more detailed explanation of our codebase.

In [1]:
import pathlib
from fpgaconvnet.parser.Parser import Parser
from fpgaconvnet.platform.Platform import Platform
from fpgaconvnet.optimiser.solvers import GreedyPartition

# load network
network_name = "single_layer"
onnx_path = f"./hardware-tutorial-assets/host-code/MNIST/{network_name}.onnx"
parser = Parser(custom_onnx=False, batch_size=1)
net = parser.onnx_to_fpgaconvnet(onnx_path)

device_path = "../../fpgaconvnet-optimiser/examples/platforms/zedboard.toml"
platform = Platform()
platform.update(device_path)
# We select port_width = 64 for solver in this example
platform.port_width   = 64

# Design Space Exploration
opt = GreedyPartition(net, platform)
opt.bram_to_lut = False
opt.off_chip_streaming = False
opt.balance_bram_uram = False
opt.rsc_allocation = 0.75
opt.transforms = []
opt.transforms_probs = []

opt.run_solver()
opt.update_partitions()

# export configuration and prediction report
pathlib.Path(network_name).mkdir(parents=True, exist_ok=True)
opt.create_report(f"{network_name}/report.json")
opt.net.save_all_partitions(f"{network_name}/config.json")

throughput = -opt.get_cost()
print(f"predicted throughput (img/s): {throughput}")
print(f"predicted resource usage: {net.partitions[0].get_resource_usage()}")
print(f"selected port_widthin solver: {platform.port_width} bits")

╔══════════════════════════════════════════════╦═════════╦══╦═══════════╦═══╦══════════════════════════════╦══════════╦══════════════════════════╦══════════╗
║ single partition (Part 1) cost (throughput): ║ 74543.4 ║  ║ slowdown: ║ 1 ║ partition optimisation time: ║ 0.04 sec ║ total optimisation time: ║ 0.04 sec ║
╚══════════════════════════════════════════════╩═════════╩══╩═══════════╩═══╩══════════════════════════════╩══════════╩══════════════════════════╩══════════╝
| COST:                     |    | RESOURCES:   |            |                |                 |              |             |              |             |
|---------------------------|----|--------------|------------|----------------|-----------------|--------------|-------------|--------------|-------------|
|                           |    | BRAM         | DSP        | LUT            | FF              | BW           | BW_IN       | BW_OUT       | BW_WEIGHT   |
| 74543.421543 (throughput) |    | 45.00/280    | 100.00/2

In [2]:
import sys
import os
sys.path.append(os.path.abspath('../../fpgaconvnet-hls'))
print(sys.path)

# Set correct environment variables
os.environ['XILINX_VIVADO'] = '/tools/Xilinx/Vivado/2019.2'
os.environ['XILINX_VITIS_HLS'] = '/tools/Xilinx/Vitis/2019.2'  # Only set this once
os.environ['PATH'] = '/tools/Xilinx/Vitis/2019.2/bin:/tools/Xilinx/Vivado/2019.2/bin:' + os.environ['PATH']

['/root/miniconda3/envs/fpgaconvnet-tutorial/lib/python310.zip', '/root/miniconda3/envs/fpgaconvnet-tutorial/lib/python3.10', '/root/miniconda3/envs/fpgaconvnet-tutorial/lib/python3.10/lib-dynload', '', '/root/.local/lib/python3.10/site-packages', '/root/miniconda3/envs/fpgaconvnet-tutorial/lib/python3.10/site-packages', '/root/fpgaconvnet-tutorial/fpgaconvnet-optimiser', '/root/fpgaconvnet-tutorial/fpgaconvnet-model', '/root/fpgaconvnet-tutorial/fpgaconvnet-hls', '/root/fpgaconvnet-tutorial/fpgaconvnet-hls']


In [3]:
import toml
from fpgaconvnet.hls.generate.network import GenerateNetwork

with open(device_path, "r") as f:
    device = toml.load(f) 
    device_name = device["device"]["part"]
    clock_period = 1000 / device["system"]["board_frequency"]
    # port_width = device["system"]["port_width"] 

# Similarly, select port_width = 64 for HLS 
port_width = 64

print(f"device name is {device_name}")
print(f"clock period is {clock_period}")
print(f"platform maximum port width is {port_width}")
    

# generate HLS code
# gen_net = GenerateNetwork(network_name, f"{network_name}/config.json", onnx_path, device_name)
gen_net = GenerateNetwork(network_name, f"{network_name}/config.json", onnx_path, device_name, clock_period, port_width)
gen_net.create_partition_project(0)

device name is xc7z020clg484-1
clock period is 5.0
platform maximum port width is 64
FIXME: use HLS backend in Parser
--------- Generating layer Conv_0 ---------
--------- Generating layer Conv_0_squeeze_Relu_1 ---------
--------- Generating layer Relu_1 ---------
--------- Generating layer squeeze_Relu_1 ---------
Writing weights stream to .dat file
Streams:1, Ports:1, PortWidth:64, DataWidth:16, Maximum stream number: 4.0, Number of elements = 400
Writing biases stream to .dat file
Streams:1, Ports:1, PortWidth:64, DataWidth:30, Maximum stream number: 2.1333333333333333, Number of elements = 16

****** Vivado(TM) HLS - High-Level Synthesis from C, C++ and SystemC v2019.2 (64-bit)
  **** SW Build 2708876 on Wed Nov  6 21:39:14 MST 2019
  **** IP Build 2700528 on Thu Nov  7 00:09:20 MST 2019
    ** Copyright 1986-2019 Xilinx, Inc. All Rights Reserved.

source /tools/Xilinx/Vivado/2019.2/scripts/vivado_hls/hls.tcl -notrace
INFO: [HLS 200-10] Running '/tools/Xilinx/Vivado/2019.2/bin/unwr

In [4]:
import numpy as np
# generate random input image, you can replace it with a fixed image
input_image = np.random.rand(1,1,28,28).astype(np.float32)
# run C simulation
gen_net.run_testbench(0, input_image)

Writing input stream to .dat file
Streams:1, Ports:1, PortWidth:64, DataWidth:16, Maximum stream number: 4.0, Number of elements = 784
Writing valid output stream to .dat file
Streams:4, Ports:1, PortWidth:64, DataWidth:16, Maximum stream number: 4.0, Number of elements = 9216

****** Vivado(TM) HLS - High-Level Synthesis from C, C++ and SystemC v2019.2 (64-bit)
  **** SW Build 2708876 on Wed Nov  6 21:39:14 MST 2019
  **** IP Build 2700528 on Thu Nov  7 00:09:20 MST 2019
    ** Copyright 1986-2019 Xilinx, Inc. All Rights Reserved.

source /tools/Xilinx/Vivado/2019.2/scripts/vivado_hls/hls.tcl -notrace
INFO: [HLS 200-10] Running '/tools/Xilinx/Vivado/2019.2/bin/unwrapped/lnx64.o/vivado_hls'
INFO: [HLS 200-10] For user 'root' on host 'b986bf97e8bd' (Linux_x86_64 version 6.8.0-49-generic) on Sat Feb 15 06:51:30 +0000 2025
INFO: [HLS 200-10] In directory '/root/fpgaconvnet-tutorial/tutorial/1_simple_end_to_end'
Sourcing Tcl script '/root/fpgaconvnet-tutorial/fpgaconvnet-hls/fpgaconvnet/hl

In [ ]:
# run C synthesis and export IP
gen_net.generate_partition_hardware(0)


****** Vivado(TM) HLS - High-Level Synthesis from C, C++ and SystemC v2019.2 (64-bit)
  **** SW Build 2708876 on Wed Nov  6 21:39:14 MST 2019
  **** IP Build 2700528 on Thu Nov  7 00:09:20 MST 2019
    ** Copyright 1986-2019 Xilinx, Inc. All Rights Reserved.

source /tools/Xilinx/Vivado/2019.2/scripts/vivado_hls/hls.tcl -notrace
INFO: [HLS 200-10] Running '/tools/Xilinx/Vivado/2019.2/bin/unwrapped/lnx64.o/vivado_hls'
INFO: [HLS 200-10] For user 'root' on host 'b986bf97e8bd' (Linux_x86_64 version 6.8.0-49-generic) on Fri Feb 14 16:28:42 +0000 2025
INFO: [HLS 200-10] In directory '/root/fpgaconvnet-tutorial/tutorial/1_simple_end_to_end'
Sourcing Tcl script '/root/fpgaconvnet-tutorial/fpgaconvnet-hls/fpgaconvnet/hls/scripts/hls/run_csynth.tcl'
INFO: [HLS 200-10] Opening project '/root/fpgaconvnet-tutorial/tutorial/1_simple_end_to_end/partition_0'.
INFO: [HLS 200-10] Opening solution '/root/fpgaconvnet-tutorial/tutorial/1_simple_end_to_end/partition_0/solution'.
INFO: [SYN 201-201] Settin

In [6]:
gen_net.run_cosimulation(0, input_image)

Writing input stream to .dat file
Streams:1, Ports:1, PortWidth:64, DataWidth:16, Maximum stream number: 4.0, Number of elements = 784
Writing valid output stream to .dat file
Streams:4, Ports:1, PortWidth:64, DataWidth:16, Maximum stream number: 4.0, Number of elements = 9216



****** Vivado(TM) HLS - High-Level Synthesis from C, C++ and SystemC v2019.2 (64-bit)
  **** SW Build 2708876 on Wed Nov  6 21:39:14 MST 2019
  **** IP Build 2700528 on Thu Nov  7 00:09:20 MST 2019
    ** Copyright 1986-2019 Xilinx, Inc. All Rights Reserved.

source /tools/Xilinx/Vivado/2019.2/scripts/vivado_hls/hls.tcl -notrace
INFO: [HLS 200-10] Running '/tools/Xilinx/Vivado/2019.2/bin/unwrapped/lnx64.o/vivado_hls'
INFO: [HLS 200-10] For user 'root' on host 'b986bf97e8bd' (Linux_x86_64 version 6.8.0-49-generic) on Fri Feb 14 15:59:11 +0000 2025
INFO: [HLS 200-10] In directory '/root/fpgaconvnet-tutorial/tutorial/1_simple_end_to_end'
Sourcing Tcl script '/root/fpgaconvnet-tutorial/fpgaconvnet-hls/fpgaconvnet/hls/scripts/hls/run_cosim.tcl'
INFO: [HLS 200-10] Opening project '/root/fpgaconvnet-tutorial/tutorial/1_simple_end_to_end/partition_0'.
INFO: [HLS 200-10] Adding test bench file 'partition_0/data/Conv_0_weights_0.dat' to the project
INFO: [HLS 200-10] Adding test bench file 'par

## Troubleshooting 
1. In `Module.py` and `Module3D.py` from `/fpgaconvnet-model/fpgaconvnet/models/modules/`, comment out the line:
   ```python
   # from xgboost import XGBRegressor
   ```
2. If the version of numpy cannot compile, you may need to uninstall numpy and reinstall a downgraded version. Version 2.0.0 is recommended. 
3. You may be working on a higher version of Linux OS that doesn't support Vivado 2019.1. In this case, Vivado may fail to simulate. Try deleting the ld folder at: `/Xilinx/Vivado/2019.1/tps/lnx64/binutils-2.26/bin/ld`
## What's next
Now that an ip has been generated, you may proceed to `hardware-tutorial.ipynb` to integrate the IP to a project. 